In [183]:
import pandas as pd
from configurations import basic_parameters as bp

In [184]:
csv_dataframe = bp['dataframe_dir'] + '/' + bp['dataframe_file']

In [185]:
names = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

df = pd.read_csv(csv_dataframe, names=names, usecols=['Open time', 'Open', 'High', 'Low', 'Close'])

In [186]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [187]:
def stochastic(data, k_window, d_window, window):
    
    # input to function is one column from df
    # containing closing price or whatever value we want to extract K and D from
    
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()
    
    stoch = ( (data - min_val) / (max_val - min_val) ) * 100
    
    K = stoch.rolling(window=k_window, center=False).mean() 
    #K = stoch
    
    D = K.rolling(window=d_window, center=False).mean() 


    return K, D

In [188]:
df['RSI'] = computeRSI(df['Close'], 14)
df['K'], df['D'] = stochastic(df['RSI'], 3, 3, 14)

In [189]:
df['time UTC'] = pd.to_datetime(df['Open time'], unit='ms', origin='unix')
df.dropna(inplace=True)
df

,Open time,Open,High,Low,Close,RSI,K,D,time UTC
31,1677715231000,23633.57,23634.82,23632.96,23634.12,60.781924,70.630393,56.267568,2023-03-02 00:00:31
32,1677715232000,23633.59,23634.14,23632.32,23633.13,56.039172,63.861023,64.376390,2023-03-02 00:00:32
33,1677715233000,23632.28,23633.18,23632.00,23633.18,56.224954,38.564245,57.685220,2023-03-02 00:00:33
34,1677715234000,23632.96,23632.96,23632.02,23632.05,50.981232,15.458772,39.294680,2023-03-02 00:00:34
35,1677715235000,23632.05,23633.49,23631.80,23633.49,56.543297,22.415855,25.479624,2023-03-02 00:00:35
...,...,...,...,...,...,...,...,...,...
86331,1677801595000,23465.54,23465.64,23464.80,23465.45,54.189816,61.018658,53.889356,2023-03-02 23:59:55
86332,1677801596000,23465.43,23465.46,23465.03,23465.39,53.553541,75.687165,64.562074,2023-03-02 23:59:56
86333,1677801597000,23465.04,23466.34,23465.04,23466.34,61.301373,79.728506,72.144776,2023-03-02 23:59:57
86334,1677801598000,23466.23,23466.23,23465.65,23465.65,54.226058,69.225085,74.880252,2023-03-02 23:59:58


In [190]:
df.loc[(df['D'] <= 0) & (df['RSI'] < 40), 'buy?'] = True
df.loc[df['buy?'] == True]

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?
198,1677715398000,23628.54,23628.79,23627.95,23628.37,31.507272,0.0,0.0,2023-03-02 00:03:18,True
796,1677715996000,23645.01,23645.54,23641.92,23642.25,18.927554,0.0,0.0,2023-03-02 00:13:16,True
797,1677715997000,23642.08,23642.08,23638.27,23641.55,18.107212,0.0,0.0,2023-03-02 00:13:17,True
798,1677715998000,23641.53,23641.56,23640.52,23641.00,17.466653,0.0,0.0,2023-03-02 00:13:18,True
799,1677715999000,23640.53,23640.99,23639.62,23640.85,17.287038,0.0,0.0,2023-03-02 00:13:19,True
...,...,...,...,...,...,...,...,...,...,...
84686,1677799949000,23457.68,23457.85,23456.59,23457.12,29.534980,0.0,0.0,2023-03-02 23:32:29,True
84687,1677799950000,23457.10,23457.10,23454.33,23454.92,20.839077,0.0,0.0,2023-03-02 23:32:30,True
84845,1677800108000,23434.54,23434.54,23430.32,23431.23,14.831080,0.0,0.0,2023-03-02 23:35:08,True
84846,1677800109000,23431.23,23431.96,23430.00,23430.10,13.473957,0.0,0.0,2023-03-02 23:35:09,True


In [197]:
rolling_seconds = 80

df['min_val'] = df['Low'].shift(-rolling_seconds).rolling(window=rolling_seconds).min()
df['max_val'] = df['High'].shift(-rolling_seconds).rolling(window=rolling_seconds).max()

df['max_val_%'] = (df['max_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)
df['min_val_%'] = (df['min_val'] - df['Open'].shift(-1)) * 100 / df['Open'].shift(-1)

#df.dropna(inplace=True)
df[:50]

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
31,1677715231000,23633.57,23634.82,23632.96,23634.12,60.781924,70.630393,56.267568,2023-03-02 00:00:31,NaN,NaN,NaN,NaN,NaN
32,1677715232000,23633.59,23634.14,23632.32,23633.13,56.039172,63.861023,64.376390,2023-03-02 00:00:32,NaN,NaN,NaN,NaN,NaN
33,1677715233000,23632.28,23633.18,23632.00,23633.18,56.224954,38.564245,57.685220,2023-03-02 00:00:33,NaN,NaN,NaN,NaN,NaN
34,1677715234000,23632.96,23632.96,23632.02,23632.05,50.981232,15.458772,39.294680,2023-03-02 00:00:34,NaN,NaN,NaN,NaN,NaN
35,1677715235000,23632.05,23633.49,23631.80,23633.49,56.543297,22.415855,25.479624,2023-03-02 00:00:35,NaN,NaN,NaN,NaN,NaN
36,1677715236000,23633.58,23633.89,23632.75,23633.16,55.003028,24.776627,20.883751,2023-03-02 00:00:36,NaN,NaN,NaN,NaN,NaN
37,1677715237000,23633.57,23633.99,23632.75,23633.99,58.094963,43.167364,30.119948,2023-03-02 00:00:37,NaN,NaN,NaN,NaN,NaN
38,1677715238000,23633.17,23634.41,23632.88,23634.03,58.243877,47.563779,38.502590,2023-03-02 00:00:38,NaN,NaN,NaN,NaN,NaN
39,1677715239000,23634.28,23639.57,23634.28,23636.02,64.922332,70.499784,53.743642,2023-03-02 00:00:39,NaN,NaN,NaN,NaN,NaN
40,1677715240000,23636.01,23636.73,23635.67,23636.22,65.519223,85.442381,67.835314,2023-03-02 00:00:40,NaN,NaN,NaN,NaN,NaN


In [192]:
tpsl = 0.075
# 0.075 RSI 40 total profit 2.47

In [193]:
sl = df.loc[(df['buy?'] == True) & (df['min_val_%'] < -tpsl) & (df['max_val_%'] < tpsl)]
sl

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
2893,1677718094000,23656.78,23657.52,23654.29,23654.65,26.277183,0.0,0.0,2023-03-02 00:48:14,True,23629.04,23656.26,0.005200,-0.109871
2894,1677718095000,23655.03,23655.11,23654.05,23654.05,24.778579,0.0,0.0,2023-03-02 00:48:15,True,23629.04,23656.26,0.009639,-0.105437
2895,1677718096000,23653.98,23654.32,23652.64,23653.68,23.874357,0.0,0.0,2023-03-02 00:48:16,True,23629.04,23656.26,0.008328,-0.106746
6662,1677721863000,23532.00,23532.00,23529.50,23530.24,38.319659,0.0,0.0,2023-03-02 01:51:03,True,23510.80,23530.86,0.002635,-0.082617
6663,1677721864000,23530.24,23530.67,23527.01,23528.93,33.568551,0.0,0.0,2023-03-02 01:51:04,True,23510.80,23530.86,0.008203,-0.077054
8181,1677723382000,23528.16,23528.56,23527.74,23527.74,36.179149,0.0,0.0,2023-03-02 02:16:22,True,23510.08,23534.99,0.029454,-0.076419
9779,1677724980000,23489.41,23489.99,23486.84,23487.00,24.149335,0.0,0.0,2023-03-02 02:43:00,True,23448.92,23488.08,0.002810,-0.163918
22254,1677737464000,23429.50,23429.88,23427.92,23428.26,25.175425,0.0,0.0,2023-03-02 06:11:04,True,23401.79,23432.36,0.017500,-0.112983
22274,1677737484000,23423.16,23424.27,23422.90,23423.70,23.816487,0.0,0.0,2023-03-02 06:11:24,True,23385.65,23425.00,0.005507,-0.162485
25081,1677740294000,23455.79,23455.80,23454.65,23454.93,34.894083,0.0,0.0,2023-03-02 06:58:14,True,23433.75,23454.67,0.000043,-0.089151


In [198]:
tp = df.loc[(df['buy?'] == True) & (df['max_val_%'] > tpsl) & (df['min_val_%'] > -tpsl)]
tp

,Open time,Open,High,Low,Close,RSI,K,D,time UTC,buy?,min_val,max_val,max_val_%,min_val_%
1172,1677716373000,23722.18,23722.92,23718.65,23720.66,37.645124,0.0,0.0,2023-03-02 00:19:33,True,23713.29,23745.00,0.102527,-0.031154
1173,1677716374000,23720.68,23721.81,23719.05,23720.53,37.485247,0.0,0.0,2023-03-02 00:19:34,True,23713.29,23745.00,0.105227,-0.028457
1565,1677716766000,23696.32,23696.84,23694.25,23695.53,24.795097,0.0,0.0,2023-03-02 00:26:06,True,23695.23,23715.64,0.084530,-0.001604
3832,1677719033000,23582.44,23582.80,23580.00,23581.46,27.440346,0.0,0.0,2023-03-02 01:03:53,True,23566.16,23600.50,0.080699,-0.064924
5334,1677720535000,23540.84,23541.00,23535.01,23537.00,17.362140,0.0,0.0,2023-03-02 01:28:55,True,23531.87,23563.48,0.112589,-0.021711
5335,1677720536000,23536.98,23537.00,23533.19,23535.30,15.945014,0.0,0.0,2023-03-02 01:28:56,True,23531.87,23563.48,0.119778,-0.014531
5336,1677720537000,23535.29,23536.19,23531.87,23535.26,15.912104,0.0,0.0,2023-03-02 01:28:57,True,23533.32,23563.48,0.124032,-0.004122
7255,1677722456000,23510.85,23511.64,23510.00,23511.41,37.052493,0.0,0.0,2023-03-02 02:00:56,True,23510.87,23529.82,0.078345,-0.002254
7342,1677722543000,23515.98,23516.83,23514.96,23516.42,28.429089,0.0,0.0,2023-03-02 02:02:23,True,23513.00,23534.23,0.075819,-0.014458
7343,1677722544000,23516.40,23516.99,23515.97,23516.40,28.380081,0.0,0.0,2023-03-02 02:02:24,True,23513.00,23534.23,0.076032,-0.014245


In [195]:
tp.shape[0]/sl.shape[0]

1.5344827586206897

In [196]:
tp['max_val_%'].sum()+sl['min_val_%'].sum()

3.677767635571037